In [ ]:
!pip install fastapi uvicorn google-generativeai python-dotenv  pyngrok nest-asyncio
!pip install websockets


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from google.colab import userdata
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import os
import google.generativeai as genai

import asyncio
import websockets
import nest_asyncio

user_d = userdata.get('NGROK_AUTH')
ngrok.set_auth_token(user_d)
# Apply patch
nest_asyncio.apply()

connected = set()

# async def echo(websocket, path):
#     connected.add(websocket)
#     try:
#         async for message in websocket:
#             print(f"Received: {message}")
#             for conn in connected:
#                 if conn != websocket:
#                     await conn.send(f"User says: {message}")
#     finally:
#         connected.remove(websocket)

# start_server = websockets.serve(echo, "localhost", 8765)

# asyncio.get_event_loop().run_until_complete(start_server)
# print("WebSocket server running on ws://localhost:8765")
# asyncio.get_event_loop().run_forever()

# # Expose port 8765
# public_url = ngrok.connect(8765, "tcp")
# print(f"WebSocket tunnel: {public_url}")


app = FastAPI()

# CORS for frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],  # ✅ Match your React dev server
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load Gemini API key
genai.configure(api_key=userdata.get("GEMINI_APi_KEy"))

model = genai.GenerativeModel("gemini-1.5-pro-latest")

class Message(BaseModel):
    prompt: str

@app.post("/chat")
async def chat(message: Message):
    response = model.generate_content(message.prompt)
    return {"response": response.text}

# Open a tunnel to the Uvicorn server
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the app
uvicorn.run(app, host="0.0.0.0", port=8000)